<a href="https://colab.research.google.com/github/YunHoLeee/MRIK/blob/main/%EC%9B%B9%ED%88%B0%EC%97%B0%EA%B5%AC_%EC%B5%9C%EC%A2%85_%EC%A3%BC%EC%A0%9C%EB%B6%84%EC%95%BC_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B6%94%EA%B0%80_%EB%B0%8F_%ED%86%B5%EA%B3%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 라이브러리 다운로드

In [ ]:
import requests  # HTTP 요청을 보내기 위한 라이브러리
import pickle  # 객체를 파일로 저장하고 불러오기 위한 라이브러리
import time  # 시간 관련 함수 제공 (예: 지연, 현재 시간)
import sys  # 파이썬 인터프리터와 관련된 작업을 수행하기 위한 라이브러리

# sys.setrecursionlimit(10000000)  # 재귀 호출의 한도를 설정 (주석 처리됨)
# from mpire import WorkerPool  # 병렬 처리 라이브러리 (주석 처리됨)

import pandas as pd  # 데이터 조작 및 분석을 위한 라이브러리
from tqdm.notebook import tqdm  # 루프의 진행 상황을 시각적으로 보여주는 라이브러리
from random import uniform  # 임의의 실수를 생성하는 함수
import lxml  # XML 및 HTML 문서를 처리하기 위한 라이브러리
import lxml.etree as et  # XML 및 HTML 문서를 생성하고 수정하기 위한 라이브러리
from bs4 import BeautifulSoup  # HTML 및 XML 문서를 구문 분석하기 위한 라이브러리
import glob  # 파일 경로 패턴을 처리하기 위한 라이브러리
import urllib3  # HTTP 클라이언트를 위한 라이브러리
urllib3.disable_warnings()  # urllib3에서 발생하는 경고를 비활성화
from natsort import natsorted  # 자연스러운 정렬을 위한 라이브러리
import re  # 정규 표현식 작업을 위한 라이브러리
import gdown  # Google Drive에서 파일을 다운로드하기 위한 라이브러리
from concurrent.futures import ThreadPoolExecutor  # 멀티스레딩을 위한 라이브러리
!pip install openpyxl

### 파일 업로드

In [ ]:
# 엑셀 파일 로드
from google.colab import files
uploaded = files.upload()

Saving 웹툰연구사 정리 데이터 최종(~2023) DEMO.xlsx to 웹툰연구사 정리 데이터 최종(~2023) DEMO.xlsx


### 주제분야 분류(대분류, 중분류) 추가

In [ ]:
urllib3.disable_warnings()

# 엑셀 파일에서 URL을 읽어오기
df = pd.read_excel('/content/웹툰연구사 정리 데이터 최종(~2023) DEMO.xlsx')  # 엑셀 파일명 적용
urls = df['URL']  # URL이 들어있는 열 가져오기

# 결과를 저장할 리스트 초기화
results = []

# 각 URL에 대해 스크래핑 수행
for url in tqdm(urls):
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.text, 'html.parser')

        # 연구분야 정보를 담고 있는 'subject' 클래스의 div 태그 찾기
        subject_div = soup.find('div', class_='subject')

        # 대분류와 중분류 추출
        if subject_div:
            overboxes = subject_div.find_all('div', class_='overbox ib')
            major_category = overboxes[0].find('a').text.strip() if len(overboxes) > 0 else '정보 없음'
            minor_category = overboxes[1].find('a').text.strip() if len(overboxes) > 1 else '0미분류0'
        else:
            major_category = '정보 없음'
            minor_category = '0미분류0'

        results.append({'URL': url, '대분류': major_category, '중분류': minor_category})

    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        results.append({'URL': url, '대분류': '오류 발생', '중분류': '오류 발생'})

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 기존 데이터프레임에 대분류와 중분류 정보 추가
df['대분류'] = results_df['대분류']
df['중분류'] = results_df['중분류']

# 결과를 엑셀 파일로 저장
output_file_path = '/content/updated_웹툰연구사 정리 데이터 최종(~2023) 대분류중분류 추출.xlsx'


  0%|          | 0/670 [00:00<?, ?it/s]

In [ ]:
df.to_excel(output_file_path, index=False)

# 결과를 출력
df.head()

# 결과 파일 다운로드
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 주제분야 분류 통계 추가

In [ ]:
# 엑셀 파일 로드
file_path = '/content/updated_웹툰연구사 정리 데이터 최종(~2023) 대분류중분류 추가.xlsx'
df = pd.read_excel(file_path)

# 대분류 통계
summary_major = df['대분류'].value_counts().reset_index()
summary_major.columns = ['대분류', '논문 수']

# 중분류 통계
summary_minor = df.groupby(['대분류', '중분류']).size().reset_index(name='논문 수')

# 엑셀 파일 저장
output_file_path = '/content/summary_웹툰연구사 정리 데이터 최종(~2023)_주제분야통계.xlsx'

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    summary_major.to_excel(writer, sheet_name='Sheet2', index=False)
    summary_minor.to_excel(writer, sheet_name='Sheet3', index=False)

# 생성된 파일 다운로드
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 엑셀 파일 로드
file_path = '/content/updated_웹툰연구사 정리 데이터 최종(~2023) 대분류중분류 추가.xlsx'
df = pd.read_excel(file_path)

# 발행기관명 통계
summary_institution = df['발행기관명'].value_counts().reset_index()
summary_institution.columns = ['발행기관명', '논문 수']

# 학술지명 통계 (발행기관명 포함)
summary_journal = df.groupby(['발행기관명', '학술지명']).size().reset_index(name='논문 수')

# 엑셀 파일 저장
output_file_path = '/content/summary_웹툰연구사 정리 데이터 최종(~2023)_기관 및 학술지.xlsx'

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    summary_institution.to_excel(writer, sheet_name='Sheet2', index=False)
    summary_journal.to_excel(writer, sheet_name='Sheet3', index=False)

# 생성된 파일 다운로드
from google.colab import files
files.download(output_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 발행연도별 통계

In [ ]:
import pandas as pd

# 엑셀 파일 로드
file_path = '/content/updated_웹툰연구사 정리 데이터 최종(~2023) 대분류중분류 추가.xlsx'
df = pd.read_excel(file_path)

# 발행연도 통계
summary_year = df['발행연도'].value_counts().reset_index()
summary_year.columns = ['발행연도', '논문 수']
summary_year = summary_year.sort_values(by='발행연도')

# 엑셀 파일 저장
output_file_path = '/content/summary_웹툰연구사 정리 데이터 최종(~2023)_연도별 통계.xlsx'

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)
    summary_year.to_excel(writer, sheet_name='Sheet2', index=False)

# 생성된 파일 다운로드
from google.colab import files
files.download(output_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 학술지별 논문 수 통계

서로 다른 발행기관에서 발행한 동명의 학술지가 있어 통계에 오차가 발생하므로, 발행기관과 학술지를 모두 활용하여 통계 도출

In [ ]:
import pandas as pd

# 엑셀 파일 로드
file_path = '/content/summary_웹툰연구사 정리 데이터 최종(~2023)_기관 및 학술지.xlsx'
df = pd.read_excel(file_path)

# 엑셀 파일의 시트 이름 확인
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)  # 시트 이름을 출력하여 확인

# 발행기관명과 학술지명을 결합하여 고유한 학술지명 생성
df['고유 학술지명'] = df['발행기관명'] + ' - ' + df['학술지명']

# 고유 학술지명별 논문 수 집계
journal_counts = df['고유 학술지명'].value_counts().reset_index()
journal_counts.columns = ['고유 학술지명', '논문 수']

# 논문 수 구간 설정 및 구간별 빈도 계산
summary_counts = {
    '논문 수 구간': ['1개', '2개', '3개', '4개', '5개', '6개', '7개 이상'],
    '학술지 수': [
        (journal_counts['논문 수'] == 1).sum(),
        (journal_counts['논문 수'] == 2).sum(),
        (journal_counts['논문 수'] == 3).sum(),
        (journal_counts['논문 수'] == 4).sum(),
        (journal_counts['논문 수'] == 5).sum(),
        (journal_counts['논문 수'] == 6).sum(),
        (journal_counts['논문 수'] >= 7).sum()
    ]
}

summary_counts_df = pd.DataFrame(summary_counts)

# 결과를 엑셀 파일로 저장
output_file_path = '논문수_구간별_학술지수.xlsx'

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    summary_counts_df.to_excel(writer, sheet_name='Sheet1', index=False)

# 생성된 파일 다운로드
from google.colab import files
files.download(output_file_path)

['Sheet1', 'Sheet2', 'Sheet3']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install gspread
!pip install pandas
!pip install gdown
!pip install openpyxl

In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

# 인증
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

IllegalCharacterError: 본 연구는 웹툰 미생의 웹드라마와 TV드라마로의 재매개 사례에 주목하여, 서사의 변형과 미디어 특성간의 관련성과 디지털서사의 특성을 살펴보았다. 분석 결과 작은 화면과 이동성, 짧은 상영시간을 특징으로 갖는 웹드라마는 이야기 차원의 통합체축에서는 에피소드의 특정과 단일 장르와 플롯⋅주제의 일반화를, 계열체축에서는 등장 인물의 세밀한 심리 묘사⋅관계의 단순화⋅시공간 한정을 통한 단순화와 압축화가 나타났다. 담화 차원에서는 연출자의 의도적 연출과 단편영화 같은 서사로 변형되었다. 반면 고화질의 대형 스크린과 시리즈물 형식인 드라마는 이야기의 통합체축에서는 에피소드 확대와 복합장르⋅멀티 플롯⋅주제의 입체화를, 계열체축에서는 명확한 인물 설정과 복잡한 인물 관계⋅극적 대립구도⋅충실한 시공간 재현⋅스펙터클한 배경을 통한 복잡하고 확장된 이야기로 변화되었다. 담화 차원에서는 반복적인 인물의 시선, 의도성과 자연스러움이 조화된 연출 방식으로 하이엔드 드라마⋅영화 같은 서사 변형을 보였다. 한편 재매개 과정에서 디지털 서사는 트랜스미디어 스토리텔링의 가능성과, 다중매체성과 표현력 확장으로 담화 차원의 개선이 나타났다. cannot be used in worksheets.

In [ ]:
# Google Sheets 파일 불러오기
spreadsheet_id = '1zSQmuXPDgIP8dNm2F6_x3AtksZY_SFJlrrsa4rAMSuo'
spreadsheet = gc.open_by_key(spreadsheet_id)

# '보완 서지정보' 시트 불러오기
worksheet = spreadsheet.worksheet('보완 서지정보')

# F열(학술지명)과 J열(발행기관명)만 가져오기
journal_names = worksheet.col_values(6)  # F열이 6번째 열
institutions = worksheet.col_values(10)  # J열이 10번째 열

# 첫 번째 행(헤더)을 제외한 데이터 결합
unique_journal_names = ["고유 학술지명"] + [f"{inst} - {journal}" for inst, journal in zip(institutions[1:], journal_names[1:])]

# Q열(17번째 열)에 데이터 업데이트
worksheet.update('Q1:Q' + str(len(unique_journal_names)), [[name] for name in unique_journal_names])

<ipython-input-46-e5c2102259a3>:16: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('Q1:Q' + str(len(unique_journal_names)), [[name] for name in unique_journal_names])


{'spreadsheetId': '1zSQmuXPDgIP8dNm2F6_x3AtksZY_SFJlrrsa4rAMSuo',
 'updatedRange': "'보완 서지정보'!Q1:Q671",
 'updatedRows': 671,
 'updatedColumns': 1,
 'updatedCells': 671}

In [ ]:
# Q열(고유 학술지명)과 K열(발행연도)만 가져오기
unique_journal_names = worksheet.col_values(17)  # Q열이 17번째 열
publish_years = worksheet.col_values(11)  # K열이 11번째 열

# 첫 번째 행(헤더)을 제외한 데이터 가져오기
unique_journal_names = unique_journal_names[1:]
publish_years = publish_years[1:]

# 데이터프레임으로 변환
df = pd.DataFrame({
    '고유 학술지명': unique_journal_names,
    '발행연도': publish_years
})

# 발행연도별 고유 학술지명 수 집계
yearly_journal_counts = df.groupby('발행연도')['고유 학술지명'].nunique().reset_index()
yearly_journal_counts.columns = ['발행연도', '논문 발행 학술지 수']

# 결과를 출력
print(yearly_journal_counts)

# 결과를 엑셀 파일로 저장
output_file_path = 'yearly_journal_counts.xlsx'
yearly_journal_counts.to_excel(output_file_path, index=False)

# 생성된 파일 다운로드
from google.colab import files
files.download(output_file_path)

    발행연도  논문 발행 학술지 수
0   2003            1
1   2004            1
2   2006            1
3   2007            5
4   2008            2
5   2009            5
6   2010            3
7   2011            9
8   2012           11
9   2013            7
10  2014           19
11  2015           28
12  2016           26
13  2017           39
14  2018           45
15  2019           48
16  2020           44
17  2021           51
18  2022           62
19  2023           70


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>